In [1]:
from collections import deque

In [2]:
class Broadcaster:
    def __init__(self) -> None:
        self.after = []
    
    def conn_to(self, targets):
        self.after.extend(targets)
    
    def conn_from(self, source):
        pass

    def pulse(self, source, hilo):
        return [(x, hilo) for x in self.after]
    
    def get_state(self):
        return 'b'
    
class FlipFlop:
    def __init__(self) -> None:
        self.state = 'off'
        self.after = []
    
    def conn_to(self, targets):
        self.after.extend(targets)
    
    def conn_from(self, source):
        pass

    def pulse(self, source, hilo):
        if hilo == 'high':
            return []
        out = 'high' if self.state == 'off' else 'low'
        self.state = 'on' if self.state == 'off' else 'off'
        return [(x, out) for x in self.after]
    
    def get_state(self):
        return self.state

class Conjunction:
    def __init__(self) -> None:
        self.prev = {}
        self.after = []
    
    def conn_to(self, targets):
        self.after.extend(targets)
    
    def conn_from(self, source):
        self.prev[source] = 'low'

    def pulse(self, source, hilo):
        self.prev[source] = hilo
        if 'low' in self.prev.values():
            out = 'high'
        else:
            out = 'low'
        return [(x, out) for x in self.after]
    
    def get_state(self):
        return str(self.prev.values())

# Part 1

Parse the input twice. First round we create all module objects and set their outward connections. On the second round we set the incoming connections for the conjection modules.

In [3]:
modules = {}

filepath = "./data/day20.txt"

with open(filepath) as f:
    while line := f.readline():
        line = line.rstrip().split(' -> ')
        if line[0][0] == '%':
            m = line[0][1:]
            modules[m] = FlipFlop()
        elif line[0][0] == '&':
            m = line[0][1:]
            modules[m] = Conjunction()
        else:
            m = line[0]
            modules[m] = Broadcaster()
        targets = line[1].replace(',', '').split(' ')
        modules[m].conn_to(targets)

with open(filepath) as f:
    while line := f.readline():
        line = line.rstrip().split(' -> ')
        if line[0][0] == '%':
            m = line[0][1:]
        elif line[0][0] == '&':
            m = line[0][1:]
        else:
            m = line[0]
        targets = line[1].replace(',', '').split(' ')
        for t in targets:
            if t in modules:
                modules[t].conn_from(m)

In [4]:
modules.keys()

dict_keys(['pr', 'jg', 'mg', 'mq', 'db', 'dx', 'bd', 'qj', 'xs', 'xd', 'gb', 'nt', 'ht', 'rh', 'sq', 'tt', 'dh', 'rz', 'cx', 'zq', 'jm', 'lj', 'mp', 'dz', 'fz', 'hj', 'broadcaster', 'zc', 'pj', 'bn', 'mr', 'mj', 'gg', 'sh', 'bf', 'hf', 'bm', 'bk', 'pq', 'xf', 'th', 'fx', 'ff', 'xr', 'bq', 'zz', 'gz', 'zs', 'vd', 'vk', 'cv', 'cd', 'zg', 'gd', 'ql', 'lt', 'ds', 'vp'])

With these classes simulating the pulse is extremely simple:

In [5]:
def run_pulse():
    lows = 0
    highs = 0
    q = deque()
    q.append(('button', 'broadcaster', 'low'))
    while q:
        source, target, hilo = q.pop()
        if hilo == 'low':
            lows += 1
        else:
            highs += 1
        if target not in modules:
            continue
        ls = modules[target].pulse(source, hilo)
        for new_target, new_hilo in ls:
            q.appendleft((target, new_target, new_hilo))
    return (lows, highs)

For part 1 the number of rounds is low enough that we can simply run the simulation 1000 times to get our answer.

In [6]:
lows, highs = 0, 0
for _ in range(1000):
    a, b = run_pulse()
    lows += a
    highs += b
lows, highs, lows*highs

(17827, 46109, 821985143)

# Part 2

Reset states by just reparsing input.

In [7]:
modules = {}

filepath = "./data/day20.txt"

with open(filepath) as f:
    while line := f.readline():
        line = line.rstrip().split(' -> ')
        if line[0][0] == '%':
            m = line[0][1:]
            modules[m] = FlipFlop()
        elif line[0][0] == '&':
            m = line[0][1:]
            modules[m] = Conjunction()
        else:
            m = line[0]
            modules[m] = Broadcaster()
        targets = line[1].replace(',', '').split(' ')
        modules[m].conn_to(targets)

with open(filepath) as f:
    while line := f.readline():
        line = line.rstrip().split(' -> ')
        if line[0][0] == '%':
            m = line[0][1:]
        elif line[0][0] == '&':
            m = line[0][1:]
        else:
            m = line[0]
        targets = line[1].replace(',', '').split(' ')
        for t in targets:
            if t in modules:
                modules[t].conn_from(m)

If it was possible to get the answer by simply running the simulation until we see that `rx` received a low pulse, this function would do it.

In [8]:
def run_pulse2():
    q = deque()
    q.append(('button', 'broadcaster', 'low'))
    while q:
        source, target, hilo = q.pop()
        if target == 'rx' and hilo == 'low':
            return True
        if target not in modules:
            continue
        ls = modules[target].pulse(source, hilo)
        for new_target, new_hilo in ls:
            q.appendleft((target, new_target, new_hilo))
    return False

Running simulations with the above run_pulse2 never finishes. Time for manual checking of what is going on.

Looking at the input, the only module that pulses to `rx` is the conjunction module `mg`. It's previous modules are:

In [9]:
modules['mg'].prev

{'jg': 'low', 'rh': 'low', 'jm': 'low', 'hf': 'low'}

In [10]:
for label in ['jg', 'rh', 'jm', 'hf']:
    print(type(modules[label]))

<class '__main__.Conjunction'>
<class '__main__.Conjunction'>
<class '__main__.Conjunction'>
<class '__main__.Conjunction'>


So let's try looking for some loops. Let's see if these modules pulse high beams to `mg` in a way that is a simple loop.

In [11]:
def run_pulse3():
    monitor = ['jg', 'rh', 'jm', 'hf']
    res = {x:[] for x in monitor}
    q = deque()
    q.append(('button', 'broadcaster', 'low'))
    while q:
        source, target, hilo = q.pop()
        if source in monitor:
            res[source].append(hilo)
        if target not in modules:
            continue
        ls = modules[target].pulse(source, hilo)
        for new_target, new_hilo in ls:
            q.appendleft((target, new_target, new_hilo))
    return res

In [12]:
hi_pulses = {x:[] for x in ['jg', 'rh', 'jm', 'hf']}
for i in range(100000):
    res = run_pulse3()
    for k, v in res.items():
        if 'high' in v:
            hi_pulses[k].append(i+1)

In [13]:
for k, v in hi_pulses.items():
    print(k)
    print(v[:10])
    print([v[i+1]-v[i] for i in range(10)])

jg
[3793, 7586, 11379, 15172, 18965, 22758, 26551, 30344, 34137, 37930]
[3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793]
rh
[4019, 8038, 12057, 16076, 20095, 24114, 28133, 32152, 36171, 40190]
[4019, 4019, 4019, 4019, 4019, 4019, 4019, 4019, 4019, 4019]
jm
[4003, 8006, 12009, 16012, 20015, 24018, 28021, 32024, 36027, 40030]
[4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003]
hf
[3947, 7894, 11841, 15788, 19735, 23682, 27629, 31576, 35523, 39470]
[3947, 3947, 3947, 3947, 3947, 3947, 3947, 3947, 3947, 3947]


Indeed they seem like they do. This isn't conclusive proof but let's assume that they keep looping like this forever. Under this assumption the first time they have all pulsed a high beam to `mg` is during the round $\text{lcm}(3793, 4019, 4003, 3947)$.

In [14]:
from math import lcm

lcm(3793, 4019, 4003, 3947)

240853834793347